In [1]:
import pandas as pd

In [40]:
physicians = pd.read_csv('../data/primary_care_physicians.csv')

In [21]:
physicians.head()

,FIPS,state,county,primary_care_physicians
0,1001,Alabama,Autauga,26.0
1,1003,Alabama,Baldwin,153.0
2,1005,Alabama,Barbour,8.0
3,1007,Alabama,Bibb,12.0
4,1009,Alabama,Blount,12.0


In [22]:
physicians.info

<bound method DataFrame.info of        FIPS              state      county  primary_care_physicians
0      1001            Alabama     Autauga                     26.0
1      1003            Alabama     Baldwin                    153.0
2      1005            Alabama     Barbour                      8.0
3      1007            Alabama        Bibb                     12.0
4      1009            Alabama      Blount                     12.0
...     ...                ...         ...                      ...
3225  72151        Puerto Rico     Yabucoa                      5.0
3226  72153        Puerto Rico       Yauco                     43.0
3227  78010  US Virgin Islands   St. Croix                     14.0
3228  78020  US Virgin Islands    St. John                      1.0
3229  78030  US Virgin Islands  St. Thomas                     28.0

[3230 rows x 4 columns]>

In [23]:
physicians.tail(2)

,FIPS,state,county,primary_care_physicians
3228,78020,US Virgin Islands,St. John,1.0
3229,78030,US Virgin Islands,St. Thomas,28.0


In [24]:
physicians.shape

(3230, 4)

In [41]:
physicians = physicians.loc[physicians['state'] == 'Tennessee']

In [26]:
physicians.head(2)

,FIPS,state,county,primary_care_physicians
2432,47001,Tennessee,Anderson,39.0
2433,47003,Tennessee,Bedford,15.0


In [27]:
physicians.shape

(95, 4)

The resulting dtataframe physicans has 95 rows i.e. there are 95 references of TN state.

In [28]:
physicians

,FIPS,state,county,primary_care_physicians
2432,47001,Tennessee,Anderson,39.0
2433,47003,Tennessee,Bedford,15.0
2434,47005,Tennessee,Benton,3.0
2435,47007,Tennessee,Bledsoe,1.0
2436,47009,Tennessee,Blount,90.0
...,...,...,...,...
2522,47181,Tennessee,Wayne,5.0
2523,47183,Tennessee,Weakley,18.0
2524,47185,Tennessee,White,9.0
2525,47187,Tennessee,Williamson,338.0


In [29]:
physicians['primary_care_physicians'].describe()

count     95.000000
mean      51.042105
std      129.311426
min        0.000000
25%        4.500000
50%       12.000000
75%       26.500000
max      806.000000
Name: primary_care_physicians, dtype: float64

There is a lot of variation in the number of physicians per county with 75%tiles of values
being within 26 i.e maximum number of counties has less than 26 physicans. But some have very
high values such as 806.


In [57]:
population=pd.read_csv('../data/population_by_county.csv')

In [58]:
population.head()

,FIPS,population,county,state,urban
0,17051,21565,Fayette County,ILLINOIS,Rural
1,17107,29003,Logan County,ILLINOIS,Rural
2,17165,23994,Saline County,ILLINOIS,Rural
3,17097,701473,Lake County,ILLINOIS,Urban
4,17127,14219,Massac County,ILLINOIS,Rural


In [63]:
population['county'] = population['county'].str.split(' ', expand = True)[0]

This is to use only the 1st part of county variable so as to make the name same for 
population and physicians

In [64]:
population

,FIPS,population,county,state,urban
0,17051,21565,Fayette,ILLINOIS,Rural
1,17107,29003,Logan,ILLINOIS,Rural
2,17165,23994,Saline,ILLINOIS,Rural
3,17097,701473,Lake,ILLINOIS,Urban
4,17127,14219,Massac,ILLINOIS,Rural
...,...,...,...,...,...
3197,47033,14399,Crockett,TENNESSEE,Rural
3198,47095,7401,Lake,TENNESSEE,Rural
3199,47093,461104,Knox,TENNESSEE,Urban
3200,53005,197518,Benton,WASHINGTON,Urban


In [43]:
physicians ['state']= physicians['state'].str.upper()

This is to make the state column same for both physicians and population

In [44]:
physicians

,FIPS,state,county,primary_care_physicians
2432,47001,TENNESSEE,Anderson,39.0
2433,47003,TENNESSEE,Bedford,15.0
2434,47005,TENNESSEE,Benton,3.0
2435,47007,TENNESSEE,Bledsoe,1.0
2436,47009,TENNESSEE,Blount,90.0
...,...,...,...,...
2522,47181,TENNESSEE,Wayne,5.0
2523,47183,TENNESSEE,Weakley,18.0
2524,47185,TENNESSEE,White,9.0
2525,47187,TENNESSEE,Williamson,338.0


Converted the state column into caps to match with population

In [71]:
physicians_merged=pd.merge(left = physicians,
                           right = population [['county','state','population', 'urban']], on = ['county','state'])


Removed the FIPS column from populations as otherwise it will be duplicated 
in the merged dataframe

In [72]:
physicians_merged

,FIPS,state,county,primary_care_physicians,population,urban
0,47001,TENNESSEE,Anderson,39.0,76061,Urban
1,47003,TENNESSEE,Bedford,15.0,48292,Rural
2,47005,TENNESSEE,Benton,3.0,16140,Rural
3,47007,TENNESSEE,Bledsoe,1.0,14836,Rural
4,47009,TENNESSEE,Blount,90.0,129927,Urban
...,...,...,...,...,...,...
89,47181,TENNESSEE,Wayne,5.0,16693,Rural
90,47183,TENNESSEE,Weakley,18.0,33510,Rural
91,47185,TENNESSEE,White,9.0,26800,Rural
92,47187,TENNESSEE,Williamson,338.0,225389,Urban


In [75]:
physicians_merged.loc[(physicians_merged['urban'] == 'Urban')]

,FIPS,state,county,primary_care_physicians,population,urban
0,47001,TENNESSEE,Anderson,39.0,76061,Urban
4,47009,TENNESSEE,Blount,90.0,129927,Urban
5,47011,TENNESSEE,Bradley,55.0,105749,Urban
7,47015,TENNESSEE,Cannon,3.0,14178,Urban
9,47019,TENNESSEE,Carter,22.0,56433,Urban
10,47021,TENNESSEE,Cheatham,23.0,40181,Urban
11,47023,TENNESSEE,Chester,4.0,17190,Urban
18,47037,TENNESSEE,Davidson,665.0,687488,Urban
21,47043,TENNESSEE,Dickson,19.0,52680,Urban
23,47047,TENNESSEE,Fayette,15.0,40164,Urban


In [76]:
physicians_merged['urban'].value_counts()

Rural    56
Urban    38
Name: urban, dtype: int64

There are 56 Rural and 38 Urban counties in TN

In [78]:
Nu_residents_per_physician=physicians_merged['population'] / physicians_merged['primary_care_physicians']

In [84]:
Nu_residents_per_physician

0      1950.282051
1      3219.466667
2      5380.000000
3     14836.000000
4      1443.633333
          ...     
89     3338.600000
90     1861.666667
91     2977.777778
92      666.831361
93     3178.279070
Length: 94, dtype: float64

In [99]:
Group1=Nu_residents_per_physician[(Nu_residents_per_physician<1500)]
len(Group1)

14

In [103]:
Group2=Nu_residents_per_physician[(Nu_residents_per_physician>1500) & (Nu_residents_per_physician<3500)]
len(Group2)

50

In [102]:
Group3=Nu_residents_per_physician[(Nu_residents_per_physician>3500)]
len(Group3)

30

In [ ]:
There are 14 counties with adequate number of residents per primary care physician,
50 counties with moderately inadequate supply and 30 counties with low inadequate supply.

In [ ]:
Rural counties have low inadequate supply of primary care physician. 

In [95]:
k=0
l=0
m=0
for x in Nu_residents_per_physician:
    print(x)
    if x < 1500
        k=k+1

    
        

SyntaxError: invalid syntax (<ipython-input-95-56ee91239c13>, line 6)